In [20]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils

import datetime as dt

# Stumbling around the v2 warehouse to find identifiers for KART Route 12

* more looping/interlining routes to come

In [2]:
shared_utils.gtfs_utils_v2?

Type:        module
String form: <module 'shared_utils.gtfs_utils_v2' from '/home/jovyan/data-analyses/_shared_utils/shared_utils/gtfs_utils_v2.py'>
File:        ~/data-analyses/_shared_utils/shared_utils/gtfs_utils_v2.py
Docstring:   GTFS utils for v2 warehouse


In [3]:
from calitp.tables import tbls

In [23]:
tbls.mart_transit_database.dim_gtfs_datasets() >> collect() >> filter(_.name.str.contains("Kings")) >> head(3)

,key,source_record_id,name,type,regional_feed_type,uri,future_uri,deprecated_date,data_quality_pipeline,manual_check__link_to_dataset_on_website,manual_check__accurate_shapes,manual_check__data_license,manual_check__authentication_acceptable,manual_check__stable_url,manual_check__localized_stop_tts,manual_check__grading_scheme_v1,base64_url,_is_current,_valid_from,_valid_to
115,e4562c470fe672741a94acda93746ddd,recwzZhQIinNX1KRp,Kings Schedule,schedule,None,https://kart.connexionz.net/rtt/public/utility...,None,None,True,Unknown,Yes,Missing,Yes,Yes - Vendor Website,Unknown,Unknown,aHR0cHM6Ly9rYXJ0LmNvbm5leGlvbnoubmV0L3J0dC9wdW...,True,2023-01-19 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
331,ca71fca800b4ddeeb95b3376be1749a1,recwzZhQIinNX1KRp,Kings Schedule,schedule,None,https://kart.connexionz.net/rtt/public/utility...,None,None,True,Unknown,Unknown,Unknown,Yes,Yes,Unknown,Unknown,aHR0cHM6Ly9rYXJ0LmNvbm5leGlvbnoubmV0L3J0dC9wdW...,False,2022-10-26 00:00:00+00:00,2022-11-21 23:59:59.999999+00:00
536,d58ff78d051be44f34bcbb2a168981db,recwzZhQIinNX1KRp,Kings Schedule,schedule,None,http://kart.connexionz.net/rtt/public/utility/...,None,None,True,Unknown,Unknown,Unknown,Yes,Yes,Unknown,Unknown,aHR0cDovL2thcnQuY29ubmV4aW9uei5uZXQvcnR0L3B1Ym...,False,2022-10-18 00:00:00+00:00,2022-10-25 23:59:59.999999+00:00


In [5]:
kings = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_organization(selected_date = '2023-02-07') >> filter(_.name.str.contains("Kings"))
kings

,key,date,feed_key,base64_url,gtfs_dataset_key,is_future,type,name,regional_feed_type
130,b7212792b389a0df398c7bff6559b324,2023-02-07,99fca48844312a4b68af992be4f7ee43,aHR0cHM6Ly9rYXJ0LmNvbm5leGlvbnoubmV0L3J0dC9wdW...,e4562c470fe672741a94acda93746ddd,False,schedule,Kings Schedule,None


In [7]:
kings_trips = shared_utils.gtfs_utils_v2.get_trips(selected_date= '2023-02-07', operator_feeds=kings.feed_key.to_list()) 

In [10]:
kings_trips.columns

Index(['key', 'service_date', 'feed_key', 'service_id', 'name',
       'regional_feed_type', 'trip_key', 'trip_id', 'trip_short_name',
       'direction_id', 'block_id', 'route_key', 'route_id', 'route_type',
       'route_short_name', 'route_long_name', 'route_desc', 'agency_id',
       'network_id', 'shape_array_key', 'gtfs_dataset_key', 'shape_id',
       'contains_warning_duplicate_trip_primary_key', 'n_stops',
       'n_stop_times', 'trip_first_departure_sec', 'trip_last_arrival_sec',
       'service_hours', 'contains_warning_duplicate_stop_times_primary_key',
       'contains_warning_missing_foreign_key_stop_id'],
      dtype='object')

In [16]:
kings_routes = kings_trips >> distinct(_.service_date, _.feed_key, _.name, _.route_id, _.route_key, _.route_short_name)
kings_routes

,service_date,feed_key,name,route_id,route_key,route_short_name
0,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,1,f1a854f5aa71eaf0d73472b95bf5371d,1
1,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,9,5655c71a2b091d9410a35d372eb65fc3,9
2,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,2,b3866380384848f0a3659208c8b9d8de,2
3,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,15,332b2d894cc6347e960304ed8c52eea1,15
4,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,6,27a36b900874da4399c540723890acd4,6
5,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,5,e708cb1666b3201b3fdc2108b337e575,5
6,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,7,bd6aff1d51620e560fcb724182a3f9d9,7
7,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,4,2e5ba90b503fe30ed31049aa67705230,4
8,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,12,8907271777a255870b3992251710f005,12
9,2023-02-07,99fca48844312a4b68af992be4f7ee43,Kings Schedule,20,1cee195682ec0ce3bbabf2155e1e12e7,20


In [18]:
kings_routes.loc[8, :].to_dict()

{'service_date': datetime.date(2023, 2, 7),
 'feed_key': '99fca48844312a4b68af992be4f7ee43',
 'name': 'Kings Schedule',
 'route_id': '12',
 'route_key': '8907271777a255870b3992251710f005',
 'route_short_name': '12'}

In [21]:
import test_loop_interline_rts

In [22]:
test_loop_interline_rts.LOOP_INTERLINE_RTS

[{'service_date': datetime.date(2023, 2, 7),
  'feed_key': '99fca48844312a4b68af992be4f7ee43',
  'name': 'Kings Schedule',
  'route_id': '12',
  'route_key': '8907271777a255870b3992251710f005',
  'route_short_name': '12'}]